I have used fitz and tesseract ocr method to extract the text.

In [ ]:
!pip install pytesseract pdf2image PyMuPDF Pillow PyPDF2 pdfminer.six pdfplumber

In [ ]:
!sudo apt install tesseract-ocr
!sudo apt install poppler-utils

In [ ]:
# covert all the pdf pages to an images and then extract texts from images
import fitz
import pytesseract
from PIL import Image
from concurrent.futures import ThreadPoolExecutor


doc = fitz.open('aasdf.pdf')
all_texts = ' '

# matrix is used for quality of image
def extract_text_from_page(page_number):
    page = doc[page_number]
    pix = page.get_pixmap(matrix=fitz.Matrix(10, 10))
    image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    extracted_text = pytesseract.image_to_string(image)
    return extracted_text.replace('\n\n', '|')

def process_page(page_number):
        text = extract_text_from_page(page_number)
        return text

# threadpoolexecutor helps to process pages parallelly (muliple pages processing at a time)
with ThreadPoolExecutor(max_workers= 10) as executor:
    page_numbers = range(len(doc))
    results = list(executor.map(process_page, page_numbers))

all_texts = ' '.join(results)
print(all_texts)


In [ ]:
# first extract all the texts from the pdf
# after that its extract all the texts from images

import pytesseract
from PIL import Image
import fitz
import io

docs = fitz.open('docs5.pdf')

def extract_image_text(image):
  return pytesseract.image_to_string(image)

all_texts = ''
for page_number in range(docs.page_count):
  page = docs[page_number]
  all_texts += page.get_text()
  text = page.get_images()

  for image_index, img in enumerate(page.get_images(), start=1):
    xref = img[0]
    base_image = docs.extract_image(xref=xref)
    image_bytes = base_image["image"]

    image = Image.open(io.BytesIO(image_bytes))

    all_texts += extract_image_text(image).replace("\n", " ")

all_texts

In [ ]:
# extract both images and text parallelly

from pdfminer.high_level import extract_text
from pdf2image import convert_from_path
import pytesseract

def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

def extract_images_from_pdf(pdf_path):
    images = convert_from_path(pdf_path)
    return images

def perform_ocr(image):
    return pytesseract.image_to_string(image)

def main(pdf_path):
    # Extract text
    text_content = extract_text_from_pdf(pdf_path)

    # Extract images
    images = extract_images_from_pdf(pdf_path)

    # Perform OCR on each image
    ocr_results = []
    for idx, image in enumerate(images):
        ocr_text = perform_ocr(image)
        ocr_results.append((f"Image {idx + 1}", ocr_text))

    return text_content, ocr_results

# upload pdf
pdf_path = '2-TOEFL Listening.pdf'

# Extract text and images
pdf_text, pdf_ocr_results = main(pdf_path)

for image_info, ocr_text in pdf_ocr_results:
    print(f"{image_info}:\n{ocr_text}\n")